In [9]:
!pip install streamlit

import streamlit as st
import pandas as pd
import joblib
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load data
df = pd.read_csv("ObesityDataSet.csv")

# Tampilkan beberapa baris pertama
st.write("### Beberapa Baris Pertama Data")
st.write(df.head())

# Informasi umum dataset
st.write("### Informasi Umum Dataset")
st.write(df.info())

# Deskripsi statistik
st.write("### Deskripsi Statistik")
st.write(df.describe(include='all'))

# Cek missing values
st.write("### Missing Values")
st.write(df.isnull().sum())

# Cek duplikat
st.write("### Jumlah Data Duplikat")
st.write(df.duplicated().sum())

# Visualisasi distribusi target
st.write("### Distribusi Target")
fig, ax = plt.subplots()
sns.countplot(data=df, x='NObeyesdad', ax=ax)
ax.set_title('Distribusi Kategori Obesitas')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
st.pyplot(fig)

# Deteksi outlier menggunakan boxplot
st.write("### Boxplot untuk Kolom Numerik")
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
for col in numeric_cols:
    fig, ax = plt.subplots()
    sns.boxplot(x=df[col], ax=ax)
    ax.set_title(f'Boxplot - {col}')
    st.pyplot(fig)

2025-06-14 12:23:50.509 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-14 12:23:50.510 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-14 12:23:50.522 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-14 12:23:50.538 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-14 12:23:50.539 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-14 12:23:50.542 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-14 12:23:50.572 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-14 12:23:50.575 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Age                             2097 non-null   object
 1   Gender                          2102 non-null   object
 2   Height                          2099 non-null   object
 3   Weight                          2100 non-null   object
 4   CALC                            2106 non-null   object
 5   FAVC                            2100 non-null   object
 6   FCVC                            2103 non-null   object
 7   NCP                             2099 non-null   object
 8   SCC                             2101 non-null   object
 9   SMOKE                           2106 non-null   object
 10  CH2O                            2105 non-null   object
 11  family_history_with_overweight  2098 non-null   object
 12  FAF                             2103 non-null   

2025-06-14 12:23:50.816 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-14 12:23:50.822 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-14 12:23:50.823 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-14 12:23:50.825 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-14 12:23:50.826 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-14 12:23:50.827 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
<ipython-input-9-1431377290>:40: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
2025-06-14 12:23:51.130 Thr

In [10]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.over_sampling import SMOTE

# Tangani missing values
df.fillna(method='ffill', inplace=True)

# Encode variabel kategorikal
cat_cols = df.select_dtypes(include='object').columns
le = LabelEncoder()
for col in cat_cols:
    df[col] = le.fit_transform(df[col])

# Split fitur dan label
X = df.drop('NObeyesdad', axis=1)
y = df['NObeyesdad']

# Atasi ketidakseimbangan kelas
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X, y)

# Normalisasi data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_res)

# Simpan scaler dan model
joblib.dump(scaler, "scaler.pkl")

<ipython-input-10-1279207807>:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


['scaler.pkl']

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import joblib

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_res, test_size=0.2, random_state=42)

# Definisikan model
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(),
    "KNN": KNeighborsClassifier()
}

results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results[name] = {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Report": classification_report(y_test, y_pred, output_dict=True),
        "Confusion": confusion_matrix(y_test, y_pred)
    }

# Simpan model terbaik
best_model = RandomForestClassifier()
best_model.fit(X_scaled, y_res)
joblib.dump(best_model, "rf_model.pkl")

['rf_model.pkl']

In [12]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [None, 10],
    'min_samples_split': [2, 5]
}

grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=3, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_rf = grid_search.best_estimator_
y_pred_best = best_rf.predict(X_test)

# Simpan model terbaik
joblib.dump(best_rf, "rf_tuned_model.pkl")

['rf_tuned_model.pkl']

In [13]:
# Judul aplikasi
st.title("Prediksi Kategori Obesitas")
st.write("Silakan lengkapi data diri Anda untuk mengetahui kategori obesitas.")

# Load model dan scaler
model = joblib.load("rf_tuned_model.pkl")
scaler = joblib.load("scaler.pkl")

# Input form
with st.form("input_form"):
    age = st.number_input("Umur (tahun)", min_value=1, max_value=120)
    height = st.number_input("Tinggi Badan (meter)", min_value=0.5, max_value=2.5)
    weight = st.number_input("Berat Badan (kg)", min_value=10, max_value=200)
    fcvc = st.slider("Frekuensi konsumsi sayuran per minggu", 0, 10)
    ncp = st.slider("Jumlah makan utama per hari", 0, 5)
    ch2o = st.slider("Konsumsi air per hari (gelas)", 0, 10)
    faf = st.slider("Frekuensi aktivitas fisik", 0, 7)
    tue = st.slider("Waktu menggunakan perangkat elektronik (jam)", 0, 10)
    caec = st.selectbox("Konsumsi camilan antara makanan utama", ["Never", "Sometimes", "Frequently", "Always"])
    smoke = st.selectbox("Merokok", ["No", "Yes"])
    scc = st.selectbox("Monitoring konsumsi kalori", ["No", "Yes"])
    favc = st.selectbox("Konsumsi makanan berkalori tinggi", ["No", "Yes"])
    gender = st.selectbox("Jenis Kelamin", ["Female", "Male"])
    family_history = st.selectbox("Riwayat keluarga kelebihan berat badan", ["No", "Yes"])

    submitted = st.form_submit_button("Prediksi")

if submitted:
    # Mapping input kategori ke numerik
    input_data = pd.DataFrame({
        'Age': [age],
        'Height': [height],
        'Weight': [weight],
        'FCVC': [fcvc],
        'NCP': [ncp],
        'CH2O': [ch2o],
        'FAF': [faf],
        'TUE': [tue],
        'CAEC': [caec],
        'SMOKE': [smoke],
        'SCC': [scc],
        'FAVC': [favc],
        'Gender': [gender],
        'family_history_with_overweight': [family_history]
    })

    # Encode kategori
    input_data.replace({
        'CAEC': {'Never': 0, 'Sometimes': 1, 'Frequently': 2, 'Always': 3},
        'SMOKE': {'No': 0, 'Yes': 1},
        'SCC': {'No': 0, 'Yes': 1},
        'FAVC': {'No': 0, 'Yes': 1},
        'Gender': {'Female': 0, 'Male': 1},
        'family_history_with_overweight': {'No': 0, 'Yes': 1}
    }, inplace=True)

    # Scaling
    scaled_input = scaler.transform(input_data)

    # Prediksi
    prediction = model.predict(scaled_input)[0]

    # Mappings hasil prediksi
    obesity_map = {
        0: 'Insufficient Weight',
        1: 'Normal Weight',
        2: 'Overweight Level I',
        3: 'Overweight Level II',
        4: 'Obesity Type I',
        5: 'Obesity Type II',
        6: 'Obesity Type III'
    }

    result = obesity_map.get(prediction, "Unknown")
    st.success(f"Kategori Obesitas Prediksi: **{result}**")

2025-06-14 12:24:14.934 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-14 12:24:14.938 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-14 12:24:14.941 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-14 12:24:14.943 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-14 12:24:15.010 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-14 12:24:15.012 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-14 12:24:15.013 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-14 12:24:15.014 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar